<a href="https://colab.research.google.com/github/noahvlone/sistem-pendukung-keputusan/blob/main/Sistem_Pendukung_Keputusan_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("NIM         : 4522210103")
print("Nama        : Farhan Ramadhan")
print("Mata Kuliah : Sistem Pendukung Keputusan A")

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
# Fungsi untuk menghitung bobot dari matriks perbandingan berpasangan
def hitung_bobot(matriks):
    total_per_kolom = np.sum(matriks, axis=0)
    matriks_normalisasi = matriks / total_per_kolom
    bobot = np.mean(matriks_normalisasi, axis=1)
    return bobot

In [ ]:
# Fungsi untuk menghitung Rasio Konsistensi (CI / RI)
def hitung_konsistensi(matriks, bobot):
    lambdamax = np.mean(np.sum(matriks * bobot, axis=1) / bobot)
    CI = (lambdamax - len(bobot)) / (len(bobot) - 1)
    # Nilai Acak Konsistensi (RI) untuk ukuran matriks 10x10 adalah 1.49
    RI = 1.49
    CR = CI / RI
    return CR

In [ ]:
# Mengambil data historis dari Yahoo Finance
symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMD', 'AMZN', 'META', 'NVDA', 'BABA', 'NFLX']
data = yf.download(symbols, start='2024-11-01', end='2024-11-07')
data.head()

In [ ]:
# Menghitung pengembalian harian dan volatilitas untuk setiap saham
returns = data['Adj Close'].pct_change()
volatilitas = returns.std()
return_harian = returns.mean()
volume_harian = data['Volume'].mean()

In [ ]:
# Matriks Perbandingan Berpasangan untuk Kriteria: Risiko, Return, Likuiditas, Jangka Waktu
matriks_kriteria = np.array([
    [1, 3, 5, 7],
    [1/3, 1, 3, 5],
    [1/5, 1/3, 1, 3],
    [1/7, 1/5, 1/3, 1]
])

# Hitung bobot dan cek konsistensi
bobot_kriteria = hitung_bobot(matriks_kriteria)
CR = hitung_konsistensi(matriks_kriteria, bobot_kriteria)
print("Bobot Kriteria:", bobot_kriteria)
print("Rasio Konsistensi:", CR)

In [ ]:
# Matriks Perbandingan Berpasangan 10x10 untuk Alternatif pada setiap Kriteria
matriks_risiko = np.zeros((10, 10))
matriks_return = np.zeros((10, 10))
matriks_likuiditas = np.zeros((10, 10))

# Matriks Risiko berdasarkan volatilitas
for i in range(10):
    for j in range(10):
        matriks_risiko[i][j] = volatilitas[i] / volatilitas[j]

# Matriks Return berdasarkan pengembalian rata-rata
for i in range(10):
    for j in range(10):
        matriks_return[i][j] = return_harian[i] / return_harian[j]

# Matriks Likuiditas berdasarkan volume perdagangan
for i in range(10):
    for j in range(10):
        matriks_likuiditas[i][j] = volume_harian[i] / volume_harian[j]

In [ ]:
# Hitung bobot untuk setiap alternatif pada setiap kriteria
bobot_risiko = hitung_bobot(matriks_risiko)
bobot_return = hitung_bobot(matriks_return)
bobot_likuiditas = hitung_bobot(matriks_likuiditas)

# Gabungkan bobot kriteria dan alternatif untuk mendapatkan skor akhir
skor_akhir = (
    bobot_kriteria[0] * bobot_risiko +
    bobot_kriteria[1] * bobot_return +
    bobot_kriteria[2] * bobot_likuiditas
)

In [ ]:
# Tampilkan rekomendasi berdasarkan skor akhir
print("\nSkor Akhir untuk setiap alternatif investasi:")
for alt, skor in zip(symbols, skor_akhir):
    print(f"{alt}: {skor}")

# Menentukan Top 3 saham terbaik
top_3_indices = np.argsort(skor_akhir)[-3:][::-1]  # Mengambil 3 skor tertinggi
top_3_symbols = [symbols[i] for i in top_3_indices]
top_3_scores = [skor_akhir[i] for i in top_3_indices]

print("\nTop 3 Saham Terbaik Berdasarkan Analisis:")
for i, (symbol, score) in enumerate(zip(top_3_symbols, top_3_scores), start=1):
    print(f"{i}. {symbol}: {score}")

rekomendasi = symbols[np.argmax(skor_akhir)]
print(f"\nRekomendasi Investasi Terbaik: {rekomendasi}")

In [ ]:
# Menyimpan data untuk setiap saham ke file CSV
all_data = {}  # Dictionary untuk menyimpan semua data saham

# Mengunduh data untuk setiap saham
for saham in symbols:
    data = yf.download(saham, start='2024-10-13', end='2024-10-27')
    all_data[saham] = data

# Menyimpan data ke dalam file CSV
for saham, data in all_data.items():
    data.to_csv(f'data_{saham}.csv')